<a href="https://colab.research.google.com/github/Camouflage10/disease-spread-model/blob/main/disease_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#comment out pips when uploading
!pip install ruamel.yaml

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 109 kB 5.1 MB/s 
     |████████████████████████████████| 546 kB 38.6 MB/s 


In [ ]:
import numpy as np
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Conv2D, MaxPooling2D, Flatten
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import confusion_matrix, recall_score, precision_score
import seaborn as sns
from ruamel.yaml import YAML
import json

In [ ]:
def load_params():
    "Updates FULL_PARAMS with the values in params.yaml and returns all as a dictionary"
    yaml = YAML(typ="safe")
    with open("params.yaml") as f:
        params = yaml.load(f)
    return params

In [ ]:
train = pd.read_csv ('dengue_features_train.csv')
train_labels=pd.read_csv('dengue_labels_train.csv')
test =pd.read_csv ('dengue_features_test.csv')

In [ ]:
#preprocessing

In [ ]:
#model

In [ ]:
#create metrics.json (optional metrics.txt also)
value = {
        "loss": 0,
        "rmse": 0,
    }
with open('metrics.json', 'w') as outfile:
  outfile.write(json.dumps(value))

In [ ]:
#create a graph called stats.png
plt.tight_layout()
fig1 = plt.gcf()
fig1.savefig('stats.png', dpi=100)

<Figure size 432x288 with 0 Axes>